In [ ]:
#| hide
from mhar.core import *

# mhar (Mentat/Matrix Hit and Run)

> mhar

This package constains one of the fastest polytope samplers known to date.
It only works with linear inequality restrictions, it does not accept V-representation.  
 
Peer Reviewed Paper: [Novel matrix hit and run for sampling polytopes and its GPU implementation](https://link.springer.com/article/10.1007/s00180-023-01411-y)  
Free Peer Reviwed Paper: [ Free: Novel matrix hit and run for sampling polytopes and its GPU implementation](https://github.com/sonder-art/mhar/blob/released/mhar_modified-3.pdf)  
Conference Presentation: [Youtube LACSC2021 conference](https://www.youtube.com/watch?v=o2CxnI6onts)  
Github: [Repo](https://github.com/sonder-art/mhar)


## What is Mentat-HAR?

We propose and analyze a new Markov Chain Monte Carlo algorithm that generates a uniform sample over full and non-full-dimensional polytopes. This algorithm, termed “Matrix Hit and Run” (MHAR), is a modification of the Hit and Run framework. For a polytope in (R^n) defined by m linear constraints, the regime (n^ 1 + 1/3) has a lower asymptotic cost per sample in terms of soft-O notation () than do existing sampling algorithms after a warm start. MHAR is designed to take advantage of matrix multiplication routines that require less computational and memory resources. Our tests show this implementation to be substantially faster than the hitandrun R package, especially for higher dimensions. Finally, we provide a python library based on PyTorch and a Colab notebook with the implementation ready for deployment in architectures with GPU or just CPU.

## Install

```sh
!pip install mhar
```

## How to use

You can acces to example jupyter notebooks with colab linksm they explain everything in detail:  
+ [Fully dimensional polytopes](https://github.com/sonder-art/mhar/blob/released/nbs/tutorial_full_dimensional.ipynb)  
+ [Non-fully dimensional polytoes](https://github.com/sonder-art/mhar/blob/released/nbs/tutorial_nonfull_dimensional.ipynb)  

A quick example using automatically generated polytopes from mhar

## Fully Dimensional

> $A^IX \leq b^I$


  
We will sample the unit hypercube that is defined as:  
> $n-hypercube = \{x \in R^n || x \in [-1,1]^n \} $  

Which we can represent in matrix restrictions:  
$ Ix \leq 1$  
$ -Ix \leq 1$  
Where $I$ is the identity matrix of dimension $n \times n$ 


In [1]:
import torch

In [5]:
# Find available devices (cpu or gpu)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print('Device:', device)

Device: cuda


We also need to decide the data-type `dtype` we are going to use. Depending on your necessities you can choose it, we recomend to use `64` bits for non-fully dimentional polytopes in order to maintain numerical inestability of the projections. Otherwiise the precision depends on the dimension of your polytope and speed you want.  
  
As of now `16` bit precision is only available for `gpu` and not `cpu`.

In [ ]:
# We will choose 64-bits
dtype = torch.float64

### Defining the polytope


Lets generate a sample polytope hypercube. For other polytopes you need to supply the constraints in torch tensors [see tutorial](https://github.com/sonder-art/mhar/blob/released/nbs/tutorial_full_dimensional.ipynb),and for non fully dimensional polytopes [see tutorial](https://github.com/sonder-art/mhar/blob/released/nbs/tutorial_nonfull_dimensional.ipynb)  

In [6]:
from mhar.polytope_examples import Hypercube

# Create a polytope (Hypercube)
hypercube_sim = Hypercube(3,
                      dtype=torch.float32,
                      device=device
                      )
hypercube_sim

/home/uumami/sonder-art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



Numeric Precision (dtype) torch.float32
Device: cuda
A_in: torch.Size([6, 3]) 
b_in: torch.Size([6, 1])

We will use this restrictions to define the polytope as:  
$A^Ix = [I | -I]x \leq 1 = b^I$

In [8]:
print('Equality Matrix',hypercube_sim.A_in)
print('Inequality Vector',hypercube_sim.b_in)

Equality Matrix tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.],
        [-1., -0., -0.],
        [-0., -1., -0.],
        [-0., -0., -1.]])
Inequality Vectpr tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


### Inner point(s)

In [10]:
from mhar.inner_point import ChebyshevCenter
import numpy as np

In order to start the algorithm we need at least one inner point $x_0$. If you know your inner point you can supply it to the algorithm, `mhar` also contains functions to compute one inner point using the [chebyshev center](https://en.wikipedia.org/wiki/Chebyshev_center) which finds the center of the smallest ball inside the polytope.

 `from mhar.inner_point import ChebyshevCenter`. The solver is in numpy so precision must be specified as `numpy.dtype`. It uses `linprog` from `scipy.optimize`. You can see the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html). 
  
It could also be the last points produced by a previous walk/run of the `mhar` 

In [12]:
x0 = ChebyshevCenter(polytope=hypercube_sim, # Polytope Object
                    lb=None,  # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None,  # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance=1e-4, # Tolerance for equality restrictions (A_eqx = b_eq)
                    device=device, # device used cpu or cuda
                    solver_precision=np.float32 # numpy dtype
                    )
x0


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


tensor([[-0.],
        [-0.],
        [-0.]], device='cuda:0')

If we want to manually input the inner points then it is enough to use a torch tensor of size $n \times l$. Where $l$ is ne number of inner points you want to supply. Just write them in column notation.  
  
We are going to manually add an other starting point to the one calcualted by the `chebyshev center` to show its functionality later.

In [ ]:
x0 = torch.cat([x0, 
            torch.tensor([[.5], [.5], [.5]]).to(device).to(dtype)
             ], dim=1)
x0

tensor([[-0.0000, 0.5000],
        [-0.0000, 0.5000],
        [-0.0000, 0.5000]], device='cuda:0')

Now we can proceed to sample the `hypercube`

### Walk

We are going to sample the polytope starting from the inner points we supply using the method `walk.walk`. It has the next arguments:

+ `polytope` is an object of the type `Polytope` or `NFDPolytope` that defines it. In our example hypercube is a `Polytope`, `NFDPolytope` is for npt fully dimensional objects.
+ `X0` a tensor containing the inner points to start the walks from.
+ `z` determines the number of simoultaneous `walks`. If the number of initial points supplied are less than `z`  ($ncols($ `x0` $) < $ `z`) then some points will be reused as starting points.  
+ `T` is the number of uncorrelated iterations you want. The number of total uncorrelated points produced by the algorithm is `z` $\times$ `T`, since `z` points are sampled at each iteration.  
+ `thinning` determines the number of points that we need to burn between iterations in order to get uncorrelated points. The suggested factor should be in the order of $O(n^3)$.
+ `warm` determines a thinning for warming the walks only at the beggining, after the this war the walks resumes as normal. It is used if you want to lose the dependency from the starting points.
+ `device` device where the tenros live `cpu` or `cuda`
+ `seed` for reproducibility
+ `verbosity` for printing what is going on

In [16]:
from mhar.walk import walk
X = walk(polytope=hypercube_sim,
        X0 = x0,  
        z=100, 
        T=1, 
        warm=0,
        thinning=3**3, 
        device= None, 
        seed=None,
        verbosity=2
)
print(f'Sampled points shape{X.shape}')
X

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  3   mI: 6   mE: None   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%
Sampled points shapetorch.Size([1, 3, 100])


tensor([[[-0.3338, -0.5635,  0.2466, -0.2501, -0.0603,  0.9541, -0.3222,
           0.1137, -0.8737,  0.9419, -0.2837,  0.6530,  0.6781,  0.1492,
           0.9140, -0.7611,  0.2826, -0.2357, -0.8645,  0.8267,  0.8019,
          -0.4441,  0.4958, -0.6613,  0.7441, -0.2193, -0.1340, -0.4858,
          -0.6970, -0.4413,  0.3817,  0.4279, -0.1637, -0.2693, -0.0182,
          -0.9639, -0.0314,  0.6485,  0.2268, -0.9117,  0.1562, -0.8305,
           0.9049, -0.0106,  0.8320, -0.5444, -0.1228,  0.2520, -0.5334,
           0.7400, -0.6726, -0.4790, -0.2184, -0.4502,  0.5855, -0.7953,
           0.8199,  0.6050, -0.3904,  0.4861,  0.6425, -0.9872, -0.7829,
          -0.4209, -0.9521,  0.8352,  0.6761, -0.8160,  0.0826,  0.6692,
           0.9779, -0.5642,  0.6161,  0.4993, -0.0903,  0.2990, -0.1937,
          -0.0180,  0.0693, -0.1304,  0.4572,  0.8642,  0.8992,  0.6749,
          -0.6140,  0.3952, -0.7586,  0.5815,  0.4864,  0.6536, -0.0772,
          -0.0545, -0.7530, -0.2361, -0.0823,  0.10

## Development


For development you can use `pip` files available in `environment_files`. You can choose either `dev_gpu` or `dev_cpu` depending on your current architechture. Both install all the necessary libraries for development.  

### nbdev


We use [nbdev](https://nbdev.fast.ai/tutorials/tutorial.html) for development, so after installation you can follow the tutorial [here](https://nbdev.fast.ai/tutorials/tutorial.html). Usually is enough to use the `nbdev_install_quarto` and `nbdev_install_hooks`.  

If you experience problems installing `Quarto` try to install it manually (it depends on your OS).